delete the index first if it exists

In [10]:
curl -XDELETE 'localhost:9200/my_index?pretty'

{
  "acknowledged" : true
}


configure an analyzer for a particular field

In [5]:
curl -XPUT 'localhost:9200/my_index?pretty' -H 'Content-Type: application/json' -d'
{
  "mappings": {
    "blog": {
      "properties": {
        "title": {
          "type":     "string",
          "analyzer": "english" 
        }
      }
    }
  }
}
'

{
  "acknowledged" : true,
  "shards_acknowledged" : true
}


see how the text will be analyzed

In [8]:
curl -XGET 'localhost:9200/my_index/_analyze?pretty' -H 'Content-Type: application/json' -d'
{
  "field": "title",
  "text": "I'\''m not happy about the foxes" 
}
'

{
  "tokens" : [
    {
      "token" : "i'm",
      "start_offset" : 0,
      "end_offset" : 3,
      "type" : "<ALPHANUM>",
      "position" : 0
    },
    {
      "token" : "happi",
      "start_offset" : 8,
      "end_offset" : 13,
      "type" : "<ALPHANUM>",
      "position" : 2
    },
    {
      "token" : "about",
      "start_offset" : 14,
      "end_offset" : 19,
      "type" : "<ALPHANUM>",
      "position" : 3
    },
    {
      "token" : "fox",
      "start_offset" : 24,
      "end_offset" : 29,
      "type" : "<ALPHANUM>",
      "position" : 5
    }
  ]
}


analyze the title field with both a standard and english analyzer

In [11]:
curl -XPUT 'localhost:9200/my_index?pretty' -H 'Content-Type: application/json' -d'
{
  "mappings": {
    "blog": {
      "properties": {
        "title": { 
          "type": "string",
          "fields": {
            "english": { 
              "type":     "string",
              "analyzer": "english"
            }
          }
        }
      }
    }
  }
}
'

{
  "acknowledged" : true,
  "shards_acknowledged" : true
}


input some test data

In [15]:
curl -XPUT 'localhost:9200/my_index/blog/1?pretty' -H 'Content-Type: application/json' -d'
{ "title": "I'\''m happy for this fox" }
'

{
  "_index" : "my_index",
  "_type" : "blog",
  "_id" : "1",
  "_version" : 2,
  "result" : "updated",
  "_shards" : {
    "total" : 2,
    "successful" : 2,
    "failed" : 0
  },
  "created" : false
}


In [14]:
curl -XPUT 'localhost:9200/my_index/blog/2?pretty' -H 'Content-Type: application/json' -d'
{ "title": "I'\''m not happy about my fox problem" }
'

{
  "_index" : "my_index",
  "_type" : "blog",
  "_id" : "2",
  "_version" : 1,
  "result" : "created",
  "_shards" : {
    "total" : 2,
    "successful" : 2,
    "failed" : 0
  },
  "created" : true
}


search using both analyzers

In [16]:
curl -XGET 'localhost:9200/my_index/_search?pretty' -H 'Content-Type: application/json' -d'
{
  "query": {
    "multi_match": {
      "type":     "most_fields", 
      "query":    "not happy foxes",
      "fields": [ "title", "title.english" ]
    }
  }
}
'

{
  "took" : 129,
  "timed_out" : false,
  "_shards" : {
    "total" : 5,
    "successful" : 5,
    "failed" : 0
  },
  "hits" : {
    "total" : 2,
    "max_score" : 1.1064976,
    "hits" : [
      {
        "_index" : "my_index",
        "_type" : "blog",
        "_id" : "2",
        "_score" : 1.1064976,
        "_source" : {
          "title" : "I'm not happy about my fox problem"
        }
      },
      {
        "_index" : "my_index",
        "_type" : "blog",
        "_id" : "1",
        "_score" : 0.7888139,
        "_source" : {
          "title" : "I'm happy for this fox"
        }
      }
    ]
  }
}


the standard analyzer result matches closer in the "not" document, as it should